In [282]:
import numpy as np
import pandas as pd

In [283]:
data = [
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]

### Encoding:
* Wine: $0$
* Whiskey: $1$
* Beer: $2$

In [284]:
X_train = np.array([
    [12.0, 1.5, 1],
    [5.0, 2.0, 0],
    [40.0, 0.0, 1],
    [13.5, 1.2, 1],
    [4.5, 1.8, 0],
    [38.0, 0.1, 1],
    [11.5, 1.7, 1],
    [5.5, 2.3, 0]
])
y_train = np.array([0, 2, 1, 0, 2, 1, 0, 2])

### Gini impurity:
$$Gini = 1- ∑p_i^2$$

In [285]:
def gini_impurity(y):
    classes, counts = np.unique(y, return_counts=True)
    probs = counts / len(y)
    return 1 - np.sum(probs ** 2)

### Logic behind ```best_split(X, y)```

This function finds the best feature and threshold to split the dataset, minimizing the weighted Gini impurity.

---

1. Initialize `best_gini` as $1.01$(as the worst value can be 1), `best_index`, and `best_threshold` to store the best split.
2. Then loop through all the features and all its value.
3. For each threshold:
   - Split the data into left and right subsets based on the threshold.
   - Calculate Gini impurity for both subsets.
4. Compute the weighted Gini impurity for the split:
   $$
   \text{Weighted Gini} = \frac{n_{\text{left}}}{n} \cdot Gini_{\text{left}} + \frac{n_{\text{right}}}{n} \cdot Gini_{\text{right}}
   $$
5. Update the best split if the current weighted Gini is smaller.
6. Return the feature index and threshold that give the best split.




In [286]:
def best_split(X, y):
    best_gini = 1.01
    best_index = None
    best_threshold = None

    n_samples, n_features = X.shape

    for feature_index in range(n_features):
        #print('This is feature ',feature_index)
        thresholds = np.unique(X[:, feature_index])

        for threshold in thresholds:
            left_mask = X[:, feature_index] <= threshold
            right_mask = X[:, feature_index] > threshold

            y_left = y[left_mask]
            y_right = y[right_mask]

            gini_left = gini_impurity(y_left)
            gini_right = gini_impurity(y_right)

            weighted_gini = (len(y_left) / len(y)) * gini_left + (len(y_right) / len(y)) * gini_right
            #print(weighted_gini)

            if weighted_gini <= best_gini:
                best_gini = weighted_gini
                best_index = feature_index
                best_threshold = threshold
                #print('This gini is selected',best_gini)

    return best_index, best_threshold


In [287]:
class Node :
  def __init__(self,feature_index= None, threshold=None, left= None, right=None, value =None):
    self.feature_index = feature_index
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

### Function to build the decision tree ```build_tree```
1. Get the values of number of samples, features, and number of unique labels
2. We will then define our stopping conditions
  * If number of labels becomes $1$.
  * Number of samples becomes less than $2$.
  * We have reached the ```max_depth```(5 here).
3. We will then find the best split at current depth.
4. We will then split the data into two parts
  * ```left_mask``` is where feature value is less than or equal to threshold.
  * ```right_mask``` is where feature value is more than threshold.
5. We will then call ```build_tree``` recursively on the left half and the right half of the tree.  

In [288]:
def build_tree(X, y, max_depth=5, min_samples=2, depth=0):
    num_samples, num_features = X.shape
    unique_labels = np.unique(y)
    num_labels = len(unique_labels)

    # stopping conditions
    if (num_labels == 1) or (num_samples < min_samples) or (depth >= max_depth):
        majority_class = np.bincount(y).argmax()
        return Node(value=majority_class)

    feature_index, threshold = best_split(X, y)
    #print(f"Depth {depth}: Best split at feature {feature_index} with threshold {threshold}")

    left_mask = X[:, feature_index] <= threshold
    right_mask = X[:, feature_index] > threshold

    # building tree recursively
    left = build_tree(X[left_mask], y[left_mask], max_depth, min_samples, depth + 1)
    right = build_tree(X[right_mask], y[right_mask], max_depth, min_samples, depth + 1)

    return Node(feature_index, threshold, left, right)


### Function to ```predict_sample```
1. If it is a leaf node then return the value of class it is storing(prediction)
2. If the value of required feature of the point is less than the threshold value then call the function recursively on the left child with same input point.
3. Similarly if the value of required feature of the point is more than the threshold value then call the function recursively on the right child with same input point.

In [289]:
def predict_sample(x, node):
    if node.value is not None: #leaf node
        return node.value

    if x[node.feature_index] <= node.threshold:
        return predict_sample(x, node.left)
    else:
        return predict_sample(x, node.right)

In [290]:
def predict(X, tree):
    return [predict_sample(x, tree) for x in X]

In [291]:
tree = build_tree(X_train, y_train)

test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])

predictions = predict(test_data, tree)
for prediction in predictions:
  if(prediction == 0):
    print('Wine')
  if(prediction == 1):
    print('Whiskey')
  if(prediction == 2):
    print('Beer')

Beer
Whiskey
Wine


### Using ```print_tree``` function to print the decision tree

1. **Leaf Node Check**: If the current node has a `value` (i.e., it's a leaf node), it prints a prediction (e.g., `Predict → [class_label]`) and returns.
2. **Decision Node**:
   - Retrieves the feature name or index being used at this node to split the data.
   - Prints the condition being checked at the current node (e.g., `If feature[0] <= 2.5:`).
   - Recursively calls itself on the left child node, increasing indentation.
   - Prints the `else` condition (i.e., if the feature is greater than the threshold).
   - Recursively calls itself on the right child node, again with increased indentation.

This function helps visualize the structure of a decision tree and understand the logic used for predictions.


In [292]:
def print_tree(node, feature_names=None, spacing=""):
    if node.value is not None: #leaf node
        print(spacing + f"Predict → {node.value}")
        return

    feature = f"feature[{node.feature_index}]" if feature_names is None else feature_names[node.feature_index]
    print(spacing + f"If {feature} <= {node.threshold:.3f}:")

    print_tree(node.left, feature_names, spacing + "  ")
    print(spacing + f"Else (if {feature} > {node.threshold:.3f}):")
    print_tree(node.right, feature_names, spacing + "  ")

In [293]:
features_name = ['Alcohol Content', "Sugar", "Color"]
print_tree(tree, features_name)

If Color <= 0.000:
  Predict → 2
Else (if Color > 0.000):
  If Sugar <= 0.100:
    Predict → 1
  Else (if Sugar > 0.100):
    Predict → 0


## Decision Tree Using Entropy:
1. This section we will implement the decision tree using entropy instead of gini impurity.
2. We define entropy as,
  $$
  Entropy = -∑p_i\log_2(p_i)
  $$
  where $p_i \ne 0$
3. In the ```best_split_entropy``` function we just use entropy instead of gini to find the best split.
4. Similarly in ```build_tree_entropy``` function we just use ```best_split_entropy``` instead of ```best_split``` to make the tree

In [303]:
def entropy(y):
  classes, counts = np.unique(y, return_counts=True)
  probabilities = counts / counts.sum()
  return -np.sum(probabilities * np.log2(probabilities, where=(probabilities != 0)))

In [300]:
def best_split_entropy(X, y):
    best_entropy = 100
    best_index = None
    best_threshold = None

    n_samples, n_features = X.shape

    for feature_index in range(n_features):
        #print('This is feature ',feature_index)
        thresholds = np.unique(X[:, feature_index])

        for threshold in thresholds:
            left_mask = X[:, feature_index] <= threshold
            right_mask = X[:, feature_index] > threshold

            y_left = y[left_mask]
            y_right = y[right_mask]

            entropy_left = entropy(y_left)
            entropy_right = entropy(y_right)

            weighted_entropy = (len(y_left) / len(y)) * entropy_left + (len(y_right) / len(y)) * entropy_right


            if weighted_entropy <= best_entropy:
                best_entropy = weighted_entropy
                best_index = feature_index
                best_threshold = threshold

    return best_index, best_threshold

In [301]:
def build_tree_entropy(X, y, max_depth=5, min_samples=2, depth=0):
    num_samples, num_features = X.shape
    unique_labels = np.unique(y)
    num_labels = len(unique_labels)

    # stopping conditions
    if (num_labels == 1) or (num_samples < min_samples) or (depth >= max_depth):
        majority_class = np.bincount(y).argmax()
        return Node(value=majority_class)

    feature_index, threshold = best_split_entropy(X, y)
    #print(f"Depth {depth}: Best split at feature {feature_index} with threshold {threshold}")

    left_mask = X[:, feature_index] <= threshold
    right_mask = X[:, feature_index] > threshold

    # building tree recursively
    left = build_tree(X[left_mask], y[left_mask], max_depth, min_samples, depth + 1)
    right = build_tree(X[right_mask], y[right_mask], max_depth, min_samples, depth + 1)

    return Node(feature_index, threshold, left, right)

Prediction using Entropy : We can see for the given test set we are getting same results from both Gini and Entropy.

In [304]:
tree_entropy = build_tree_entropy(X_train, y_train)

test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])

predictions = predict(test_data, tree)
for prediction in predictions:
  if(prediction == 0):
    print('Wine')
  if(prediction == 1):
    print('Whiskey')
  if(prediction == 2):
    print('Beer')

Beer
Whiskey
Wine
